In [1]:
import collections
import json
import urllib2

In [2]:
simulation = dict(
    scenarios = [
        dict(
            test_case = dict(
                familles = [
                    dict(
                        parents = ['ind0', 'ind1'],
                        enfants = ['ind2'],
                        ),
                    ],
                foyers_fiscaux = [
                    dict(
                        declarants = ['ind0'],
                        personnes_a_charge = ['ind2'],
                        ),
                    dict(
                        declarants = ['ind1'],
                        ),
                    ],
                individus = [
                    dict(
                        activite = u'Actif occupé',
                        birth = '1970-01-01',
                        id = 'ind0',
                        sali = 16000,
                        statmarit = u'Divorcé',
                        ),
                    dict(
                        activite = u'Chômeur',
                        birth = '1970-01-02',
                        id = 'ind1',
                        choi = 12500,
                        statmarit = u'Célibataire',
                        ),
                    dict(
                        activite = u'Étudiant, élève',
                        birth = '2000-01-03',
                        id = 'ind2',
                        ),
                    ],
                menages = [
                    dict(
                        personne_de_reference = 'ind0',
                        conjoint = 'ind1',
                        enfants = ['ind2'],
                        ),
                    ],
                ),
            year = 2013,
            ),
         dict(
            test_case = dict(
                familles = [
                    dict(
                        parents = ['ind0', 'ind1'],
                        enfants = ['ind2'],
                        ),
                    ],
                foyers_fiscaux = [
                    dict(
                        declarants = ['ind0'],
                        personnes_a_charge = ['ind2'],
                        ),
                    dict(
                        declarants = ['ind1'],
                        ),
                    ],
                individus = [
                    dict(
                        activite = u'Actif occupé',
                        birth = '1970-01-01',
                        id = 'ind0',
                        sali = 16000,
                        statmarit = u'Divorcé',
                        ),
                    dict(
                        activite = u'Chômeur',
                        birth = '1970-01-02',
                        id = 'ind1',
                        choi = 7000,
                        statmarit = u'Célibataire',
                        ),
                    dict(
                        activite = u'Étudiant, élève',
                        birth = '2000-01-03',
                        id = 'ind2',
                        ),
                    ],
                menages = [
                    dict(
                        personne_de_reference = 'ind0',
                        conjoint = 'ind1',
                        enfants = ['ind2'],
                        ),
                    ],
                ),
            year = 2013,
            ),
        dict(
            test_case = dict(
                familles = [
                    dict(
                        parents = ['ind0', 'ind1'],
                        enfants = ['ind2'],
                        ),
                    ],
                foyers_fiscaux = [
                    dict(
                        declarants = ['ind0'],
                        personnes_a_charge = ['ind2'],
                        ),
                    dict(
                        declarants = ['ind1'],
                        ),
                    ],
                individus = [
                    dict(
                        activite = u'Actif occupé',
                        birth = '1970-01-01',
                        id = 'ind0',
                        sali = 16000,
                        statmarit = u'Divorcé',
                        ),
                    dict(
                        activite = u'Chômeur',
                        birth = '1970-01-02',
                        id = 'ind1',
                        choi = 1000,
                        statmarit = u'Célibataire',
                        ),
                    dict(
                        activite = u'Étudiant, élève',
                        birth = '2000-01-03',
                        id = 'ind2',
                        ),
                    ],
                menages = [
                    dict(
                        personne_de_reference = 'ind0',
                        conjoint = 'ind1',
                        enfants = ['ind2'],
                        ),
                    ],
                ),
            year = 2013,
            ),
       ],
    )

In [3]:
request = urllib2.Request('http://api.openfisca.fr/api/1/simulate', headers = {
    'Content-Type': 'application/json',
    'User-Agent': 'OpenFisca-Notebook',
    })
try:
    response = urllib2.urlopen(request, json.dumps(simulation))
except urllib2.HTTPError as response:
    print response.read()
    raise
response_dict = json.loads(response.read(), object_pairs_hook = collections.OrderedDict)
tree = response_dict['value']

In [4]:
def any(iterator):
    for item in iterator:
        if item:
            return True
    return False

def iter_columns_from_tree(node, base_values = None):
    children = node.get('children')
    if children:
        child_base_values = base_values
        for child in children:
            for column in iter_columns_from_tree(child, base_values = child_base_values):
                yield column
            if child_base_values is None:
                child_base_values = child['values']
            else:
                child_base_values = [
                    base_value + value
                    for base_value, value in zip(child_base_values, child['values'])
                    ]

    if any(value != 0 for value in node['values']):
        column = dict(
            base_values = base_values or [0] * len(node['values']),
            code = node['code'],
            )
        column.update(node)
        yield column

In [5]:
for column_index, column in enumerate(iter_columns_from_tree(tree)):
    print u'{:40s} | {}'.format(column['name'], ' | '.join('{:6.0f}'.format(value) for value in column['values']))

Salaires super bruts                     |  24073 |  24073 |  24073
Cotisations sociales patronales          |  -8785 |  -8785 |  -8785
Allègement sur les bas salaires (Fillon) |   3631 |   3631 |   3631
Crédit d'impôt compétitivité-emploi      |    788 |    788 |    788
Salaires bruts                           |  19707 |  19707 |  19707
Cotisations sociales salariées           |  -2720 |  -2720 |  -2720
CSG déductible (salaires)                |   -987 |   -987 |   -987
Salaires imposables                      |  16000 |  16000 |  16000
CSG non déductible (salaires)            |   -465 |   -465 |   -465
CRDS (salaires)                          |    -97 |    -97 |    -97
Salaires nets                            |  15438 |  15438 |  15438
Revenu du travail                        |  15438 |  15438 |  15438
Chômage brut                             |  12500 |   7000 |   1000
Chômage                                  |  12500 |   7000 |   1000
Chômage net                              |  1250